In [1]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from pymfe.mfe import MFE
import numpy as np
import pandas as pd

In [3]:
mfe_kwargs = {
    "features":
            ["attr_ent", "class_ent", "eq_num_attr", "joint_ent", "mut_inf", "ns_ratio", "eigenvalues", "g_mean", "gravity", "kurtosis",
            "max", "mean", "min", "nr_cor_attr", "nr_norm", "range",
            "sd_ratio","skewness", "var", "attr_to_inst", "cat_to_num",
            "freq_class", "inst_to_attr", "nr_attr", "nr_cat", "nr_class", "nr_inst", "nr_num", "num_to_cat"],
    "summary":
            "mean"
}

In [4]:
import pickle

with open("/home/cherniak/itmo_job/GAMLET/data/no_meta_features_and_fedot_pipelines_raw/id2dataset_id.pickle", "rb") as f:
    id2dataset_id = pickle.load(f)

In [5]:
import os
from tqdm import tqdm


folds_folder = "/home/cherniak/itmo_job/datasets_folds"

meta_features = {}
for id, dataset_id in tqdm(id2dataset_id.items()):
    dataset_name, dataset_fold = dataset_id.split("_")
    x_fname = f"train_{dataset_name}_fold{dataset_fold}.npy"
    y_fname = f"trainy_{dataset_name}_fold{dataset_fold}.npy"
    x = np.load(os.path.join(folds_folder, x_fname))
    y = np.load(os.path.join(folds_folder, y_fname))
    mfe = MFE(**mfe_kwargs, num_cv_folds=1)
    mfe.fit(x, y)
    ft = mfe.extract()
    ft = {k: v for k,v in zip(*ft)}
    meta_features[dataset_id] = ft

100%|██████████| 101/101 [02:48<00:00,  1.67s/it]


In [ ]:
df = pd.DataFrame.from_dict(meta_features).T
print(df.isna().sum())
df.describe()

In [ ]:
to_drop = ["cat_to_num", "nr_cat", "num_to_cat", "nr_norm"]
df = df.drop(to_drop, axis=1)
df = df.dropna(axis=1)

In [6]:
df.to_csv("/home/cherniak/itmo_job/GAMLET/data/meta_features_and_fedot_pipelines/pymfe.csv", index=True)